# 🌤️ API Météo Python - Documentation

## 📋 Table des matières
- [Configuration]
- [ Variables d'environnement]
- [Analyse Feels Like]
- [Dataset structuré]
- [Alertes température]
- [Comparaison de villes]

---

## imports ,chargement des variable d'environement et  configuration de la clé api

In [ ]:
import requests
import json
import os
from datetime import datetime
from dotenv import load_dotenv

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

# Configuration API
API_KEY = os.getenv('OPENWEATHER_API_KEY')
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"

# Vérifier que la clé API est chargée
if not API_KEY:
    raise ValueError("ERREUR: La clé API n'est pas définie dans le fichier .env")

## Focntions principales

In [ ]:
def get_weather(city='Paris'):
    """
    Récupère les données météo d'une ville via OpenWeatherMap API
    
    Args:
        city (str): Nom de la ville (défaut: 'Paris')
        
    Returns:
        dict: Données météo au format JSON ou None en cas d'erreur
    """
    url = BASE_URL
    params = {
        'q': city,
        'appid': API_KEY,  # Utilise la variable d'environnement
        'units': 'metric',
        'lang': 'fr'
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des données pour {city}: {e}")
        return None

In [ ]:

#  Stockage dans une variable d'environnement
def store_weather(weather_data):
    if not weather_data:
        """" Stocke la condition dans une varible d'environement """
        return None
        
    weather_main = weather_data['weather'][0]['main']
    os.environ['WEATHER_CONDITION'] = weather_main
    
    print(f" Météo stockée: {weather_main}")
    return weather_main

#  Script conditionnel selon la météo
def show_weather_message():
    """ Affiche un message personnalisé selon la condition météo"""
    weather = os.getenv('WEATHER_CONDITION')
    
    print(f"\n Condition météo: {weather}")
    
    if weather == 'Rain':
        print(" Il pleut ! ")
        
    elif weather == 'Clear':
        print(" Il fait beau ! ")
        
    elif weather == 'Clouds':
        print(" Temps nuageux ")
        
    elif weather == 'Snow':
        print(" Il neige !  ")
        
    elif weather == 'Thunderstorm':
        print(" Orage ! ")
        
    else:
        print(f" Météo: {weather}")

#  Programme principal
def main():
    print("=== SCRIPT MÉTÉO  ===")
    
    # Récupération des données
    print(" Récupération de la météo...")
    data = get_weather()
    
    if not data:
        print(" Impossible de récupérer la météo")
        return
    
    # Stockage dans l'environnement  
    print(" Stockage dans variable d'environnement...")
    store_weather(data)
    
    # Affichage du message conditionnel
    print(" Affichage du message:")
    show_weather_message()
    
    print(f"\n🔍 Variable: WEATHER_CONDITION = {os.getenv('WEATHER_CONDITION')}")

# Lancer le programme
if __name__ == "__main__":
    main()

Question 3: exploiter la donnéee feel_like

In [ ]:
#  Analyser feels_like vs température
def analyze_feels_like(weather_data):
    """ Compare la température réelle au ressenti """
    if not weather_data:
        return
    
    temp = weather_data['main']['temp']
    feels_like = weather_data['main']['feels_like']
    difference = feels_like - temp
    city = weather_data['name']
    
    print(f"\n {city}:")
    print(f"   Température: {temp}°C")
    print(f"   Ressenti: {feels_like}°C")
    print(f"   Différence: {difference:+.1f}°C")
    
    # Interprétation IA
    if abs(difference) < 1:
        print("    IA: Conditions confortables")
    elif difference > 2:
        print("    IA: Recommander hydratation")
    elif difference < -2:
        print("    IA: Recommander vêtements chauds")



## Enregistrement dans une collection  structuré format Json

In [ ]:
#  Créer dataset JSON de 5 villes
def create_weather_dataset():
    """Crée un dataset JSON de 5 villes"""
    cities = ['Paris', 'London', 'Tokyo', 'New York', 'Sydney']
    dataset = []
    
    print(" Collecte données 5 villes...")
    
    for city in cities:
        data = get_weather(city)
        if data:
            city_info = {
                'ville': data['name'],
                'temperature': data['main']['temp'],
                'humidite': data['main']['humidity'],
                'etat_ciel': data['weather'][0]['main'],
                'feels_like': data['main']['feels_like']
            }
            dataset.append(city_info)
            analyze_feels_like(data)
    
    return dataset

#  Sauvegarde et analyser le dataset
def save_dataset(dataset):
    """Sauvegarde le dataset en JSON"""
    filename = 'weather_dataset.json'
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(dataset, f, indent=2, ensure_ascii=False)
    
    print(f"\n Dataset sauvegardé: {filename}")
    print("\n DATASET JSON:")
    print(json.dumps(dataset, indent=2, ensure_ascii=False))
    
    print("\n TRANSFORMATION POUR ML:")
    print(" Features: température, humidité → Prédire feels_like")
    print(" Preprocessing: Normalisation + One-hot encoding")
    print(" Modèles: Régression pour prédiction température")

#  Programme principal
def main():
    print("=== ANALYSE FEELS_LIKE + DATASET ===")
    
    # Analyse d'une ville
    print(" Analyse feels_like Paris...")
    data = get_weather('Paris')
    if data:
        os.environ['WEATHER_CONDITION'] = data['weather'][0]['main']
        analyze_feels_like(data)
    
    # Créer dataset 5 villes
    print("\n Création dataset 5 villes...")
    dataset = create_weather_dataset()
    
    # Sauvegarde
    print("\n Sauvegarde et analyse ML...")
    save_dataset(dataset)
    
    print(f"\n Variable: WEATHER_CONDITION = {os.getenv('WEATHER_CONDITION')}")

# Lancer le programme
if __name__ == "__main__":
    main()

## Enregistrement des température critique format Json

In [ ]:
from datetime import datetime

#  Vérification température critique
def check_temperature_alert(weather_data):
    """Vérifie si la température dépasse 40°C"""
    if not weather_data:
        return
    
    temp = weather_data['main']['temp']
    city = weather_data['name']
    
    print(f"\n {city}: {temp}°C")
    
    # Alerte si > 40°C
    if temp > 40:
        print(" ALERTE CANICULE!")
        print("    Buvez beaucoup d'eau")
        print("    Restez au frais")
        
        # Sauvegarder alerte
        alert = {
            'ville': city,
            'temperature': temp,
            'type': 'CANICULE',
            'heure': datetime.now().strftime('%H:%M:%S')
        }
        
        with open('alerte.json', 'w') as f:
            json.dump(alert, f, indent=2)
        
        print("    Alerte sauvegardée!")
        
    else:
        print("    Température normale")

## Création de dataset 

In [ ]:
def create_dataset():
    """Crée un dataset avec vérification des alertes"""
    cities = ['Paris', 'London', 'Tokyo', 'New York', 'Sydney']
    dataset = []
    
    for city in cities:
        data = get_weather(city)
        if data:
            city_info = {
                'ville': city,
                'temperature': data['main']['temp'],
                'humidite': data['main']['humidity'],
                'etat_ciel': data['weather'][0]['main']
            }
            dataset.append(city_info)
            check_temperature_alert(data)
    
    # Sauvegarder dataset
    with open('dataset.json', 'w') as f:
        json.dump(dataset, f, indent=2)
    
    print(f"\n Dataset sauvegardé: {len(dataset)} villes")
    return dataset

#  Programme principal
def main():
    print("=== MÉTÉO + ALERTES ===")
    
    # Test Paris
    data = get_weather('Paris')
    if data:
        os.environ['WEATHER_CONDITION'] = data['weather'][0]['main']
        check_temperature_alert(data)
    
    # Dataset 5 villes
    print("\n Collecte 5 villes...")
    dataset = create_dataset()
    
    print(f"\n Météo stockée: {os.getenv('WEATHER_CONDITION')}")


# Lancer
if __name__ == "__main__":
    main()
   

##  Comparer deux villes 

In [ ]:

#  Comparer deux villes
def compare_cities(city1, city2):
    print(f"\n COMPARAISON: {city1} vs {city2}")
    print("=" * 50)
    
    # Récupérer données
    data1 = get_weather(city1)
    data2 = get_weather(city2)
    
    if not data1 or not data2:
        print(" Impossible de comparer")
        return None
    
    # Extraire indicateurs
    temp1 = data1['main']['temp']
    temp2 = data2['main']['temp']
    humidity1 = data1['main']['humidity']
    humidity2 = data2['main']['humidity']
    pressure1 = data1['main']['pressure']
    pressure2 = data2['main']['pressure']
    
    # Affichage
    print(f" TEMPÉRATURE:")
    print(f"   {city1}: {temp1}°C")
    print(f"   {city2}: {temp2}°C")
    print(f"   → Plus chaud: {city1 if temp1 > temp2 else city2}")
    
    print(f"\n HUMIDITÉ:")
    print(f"   {city1}: {humidity1}%")
    print(f"   {city2}: {humidity2}%")
    
    print(f"\n PRESSION:")
    print(f"   {city1}: {pressure1} hPa")
    print(f"   {city2}: {pressure2} hPa")
    
    # Retourner données
    return {
        'ville1': city1,
        'ville2': city2,
        'temperature': {'ville1': temp1, 'ville2': temp2, 'diff': abs(temp1 - temp2)},
        'humidite': {'ville1': humidity1, 'ville2': humidity2, 'diff': abs(humidity1 - humidity2)}
    }



## Programme principal 

In [ ]:
#  Programme principal
def main():
    """Point d'entrée principal"""
    print("=== COMPARATEUR MÉTÉO ===")
    
    # Comparaison Alger vs Montréal
    print("\n1. Comparaison Alger vs Montréal...")
    comp1 = compare_cities('Alger', 'Montréal')
    
    # Stocker dans env
    if comp1:
        os.environ['LAST_COMPARISON'] = f"{comp1['ville1']}_vs_{comp1['ville2']}"
        print(f"\n Dernière comparaison: {os.getenv('LAST_COMPARISON')}")

# Lancer
if __name__ == "__main__":
    main()